In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!tar -xzvf /content/drive/MyDrive/nlp/hwu.tar.gz

hwu/
hwu/categories.json
hwu/train_5.csv
hwu/train_10.csv
hwu/val.csv
hwu/test.csv
hwu/train.csv


In [7]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np

print("Đang đọc dữ liệu...")
df_train = pd.read_csv('hwu/train.csv', sep=',')
df_val = pd.read_csv('hwu/val.csv', sep=',')
df_test = pd.read_csv('hwu/test.csv', sep=',')

df_train = df_train.rename(columns={'category': 'intent'})
df_val = df_val.rename(columns={'category': 'intent'})
df_test = df_test.rename(columns={'category': 'intent'})

print("Train shape:", df_train.shape)
print("Validation shape:", df_val.shape)
print("Test shape:", df_test.shape)
print("\n--- Dữ liệu mẫu (Đã sửa) ---")
print(df_train.head())

from sklearn.preprocessing import LabelEncoder

all_intents = pd.concat([df_train['intent'], df_val['intent'], df_test['intent']])
label_encoder = LabelEncoder()
label_encoder.fit(all_intents)

y_train = label_encoder.transform(df_train['intent'])
y_val = label_encoder.transform(df_val['intent'])
y_test = label_encoder.transform(df_test['intent'])

X_train = df_train['text']
X_val = df_val['text']
X_test = df_test['text']

num_classes = len(label_encoder.classes_)
print(f"\nĐã mã hóa {num_classes} lớp.")
print(f"y_train 5 mẫu đầu: {y_train[:5]}")

Đang đọc dữ liệu...
Train shape: (8954, 2)
Validation shape: (1076, 2)
Test shape: (1076, 2)

--- Dữ liệu mẫu (Đã sửa) ---
                                                text       intent
0                what alarms do i have set right now  alarm_query
1                    checkout today alarm of meeting  alarm_query
2                              report alarm settings  alarm_query
3  see see for me the alarms that you have set to...  alarm_query
4                       is there an alarm for ten am  alarm_query

Đã mã hóa 64 lớp.
y_train 5 mẫu đầu: [0 0 0 0 0]


# Nhiệm vụ 1: Pipeline TF-IDF + Logistic Regression

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, f1_score

print("\n--- [Nhiệm vụ 1] Đang thực hiện TF-IDF + Logistic Regression ---")

tfidf_lr_pipeline = make_pipeline(
    TfidfVectorizer(max_features=5000),
    LogisticRegression(max_iter=1000)
)

print("Đang huấn luyện pipeline...")
tfidf_lr_pipeline.fit(X_train, y_train)
print("...Huấn luyện xong.")

print("\n--- [Nhiệm vụ 1] Kết quả trên tập Test ---")
y_pred_lr = tfidf_lr_pipeline.predict(X_test)
print(classification_report(y_test, y_pred_lr, target_names=label_encoder.classes_))

f1_lr = f1_score(y_test, y_pred_lr, average='macro')
loss_lr = np.nan


--- [Nhiệm vụ 1] Đang thực hiện TF-IDF + Logistic Regression ---
Đang huấn luyện pipeline...
...Huấn luyện xong.

--- [Nhiệm vụ 1] Kết quả trên tập Test ---
                          precision    recall  f1-score   support

             alarm_query       0.90      0.95      0.92        19
            alarm_remove       1.00      0.73      0.84        11
               alarm_set       0.77      0.89      0.83        19
       audio_volume_down       1.00      0.75      0.86         8
       audio_volume_mute       0.92      0.80      0.86        15
         audio_volume_up       0.93      1.00      0.96        13
          calendar_query       0.45      0.53      0.49        19
         calendar_remove       0.89      0.89      0.89        19
            calendar_set       0.87      0.68      0.76        19
          cooking_recipe       0.59      0.68      0.63        19
        datetime_convert       0.67      0.75      0.71         8
          datetime_query       0.74      0.89    

# Nhiệm vụ 2: Pipeline Word2Vec (Trung bình) + Dense Layer

In [9]:
!pip install gensim -q

import numpy as np
from gensim.models import Word2Vec
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

print("\n--- [Nhiệm vụ 2] Đang thực hiện Word2Vec (Avg) + Dense Layer ---")

print("Đang huấn luyện Word2Vec...")
sentences = [str(text).split() for text in X_train]
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
print("...Huấn luyện Word2Vec xong.")

def sentence_to_avg_vector(text, model):
    words = str(text).split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]

    if not word_vectors:
        return np.zeros(model.vector_size)

    avg_vector = np.mean(word_vectors, axis=0)
    return avg_vector

X_train_avg = np.array([sentence_to_avg_vector(text, w2v_model) for text in X_train])
X_val_avg = np.array([sentence_to_avg_vector(text, w2v_model) for text in X_val])
X_test_avg = np.array([sentence_to_avg_vector(text, w2v_model) for text in X_test])

model_w2v_avg = Sequential([
    Dense(128, activation='relu', input_shape=(w2v_model.vector_size,)),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model_w2v_avg.summary()

print("\nĐang huấn luyện mô hình Keras...")
model_w2v_avg.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

early_stopping = EarlyStopping(patience=3, restore_best_weights=True)

model_w2v_avg.fit(
    X_train_avg, y_train,
    validation_data=(X_val_avg, y_val),
    epochs=20,
    batch_size=32,
    callbacks=[early_stopping],
    shuffle=True
)

print("\n--- [Nhiệm vụ 2] Kết quả trên tập Test ---")
loss_w2v_avg, acc_w2v_avg = model_w2v_avg.evaluate(X_test_avg, y_test)
print(f"Test Loss: {loss_w2v_avg:.4f}, Test Accuracy: {acc_w2v_avg:.4f}")

y_pred_w2v_nn = model_w2v_avg.predict(X_test_avg)
y_pred_w2v_nn_classes = np.argmax(y_pred_w2v_nn, axis=1)
print(classification_report(y_test, y_pred_w2v_nn_classes, target_names=label_encoder.classes_))

f1_w2v_avg = f1_score(y_test, y_pred_w2v_nn_classes, average='macro')


--- [Nhiệm vụ 2] Đang thực hiện Word2Vec (Avg) + Dense Layer ---
Đang huấn luyện Word2Vec...
...Huấn luyện Word2Vec xong.


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 128)            │        12,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,184 (82.75 KB)

 Trainable params: 21,184 (82.75 KB)

 Non-trainable params: 0 (0.00 B)


Đang huấn luyện mô hình Keras...
Epoch 1/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.0173 - loss: 4.1593 - val_accuracy: 0.0279 - val_loss: 4.0970
Epoch 2/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0355 - loss: 4.0941 - val_accuracy: 0.0548 - val_loss: 4.0151
Epoch 3/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0541 - loss: 4.0135 - val_accuracy: 0.0846 - val_loss: 3.8982
Epoch 4/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.0599 - loss: 3.9012 - val_accuracy: 0.0874 - val_loss: 3.7883
Epoch 5/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.0734 - loss: 3.8039 - val_accuracy: 0.0883 - val_loss: 3.6860
Epoch 6/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.0845 - loss: 3.7189 - val_accuracy: 0.1032 - val_loss: 3.6114
Epoch 7/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0896 - loss: 3.6418 - val_accuracy: 0.1385 - val_loss: 3.5342
Epoch 8/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.106

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Nhiệm vụ 3 & 4: Mô hình LSTM (Pre-trained và Scratch)

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM

vocab_size = 10000
max_len = 50

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<UNK>")
tokenizer.fit_on_texts(X_train)

train_sequences = tokenizer.texts_to_sequences(X_train)
val_sequences = tokenizer.texts_to_sequences(X_val)
test_sequences = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(train_sequences, maxlen=max_len, padding='post')
X_val_pad = pad_sequences(val_sequences, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(test_sequences, maxlen=max_len, padding='post')

print(f"\nĐã Tokenize và Pad. Shape X_train_pad: {X_train_pad.shape}")
vocab_size_actual = len(tokenizer.word_index) + 1

print("\n\n--- [Nhiệm vụ 3] Đang thực hiện Embedding Pre-trained + LSTM ---")

embedding_dim = w2v_model.vector_size
embedding_matrix = np.zeros((vocab_size_actual, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < vocab_size_actual and word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

print(f"Đã tạo Embedding Matrix. Shape: {embedding_matrix.shape}")

lstm_model_pretrained = Sequential([
    Embedding(
        input_dim=vocab_size_actual,
        output_dim=embedding_dim,
        weights=[embedding_matrix],
        input_length=max_len,
        trainable=False
    ),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(num_classes, activation='softmax')
])

lstm_model_pretrained.summary()

print("\nĐang huấn luyện Mô hình 3...")
lstm_model_pretrained.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

lstm_model_pretrained.fit(
    X_train_pad, y_train,
    validation_data=(X_val_pad, y_val),
    epochs=20,
    batch_size=32,
    callbacks=[early_stopping],
    shuffle=True
)

print("\n--- [Nhiệm vụ 3] Kết quả trên tập Test ---")
loss_lstm_pre, acc_lstm_pre = lstm_model_pretrained.evaluate(X_test_pad, y_test)
print(f"Test Loss: {loss_lstm_pre:.4f}, Test Accuracy: {acc_lstm_pre:.4f}")

y_pred_lstm_pre = lstm_model_pretrained.predict(X_test_pad)
y_pred_lstm_pre_classes = np.argmax(y_pred_lstm_pre, axis=1)
print(classification_report(y_test, y_pred_lstm_pre_classes, target_names=label_encoder.classes_))

f1_lstm_pre = f1_score(y_test, y_pred_lstm_pre_classes, average='macro')


print("\n\n--- [Nhiệm vụ 4] Đang thực hiện Embedding (Scratch) + LSTM ---")

embedding_dim_scratch = 100

lstm_model_scratch = Sequential([
    Embedding(
        input_dim=vocab_size_actual,
        output_dim=embedding_dim_scratch,
        input_length=max_len
    ),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(num_classes, activation='softmax')
])

lstm_model_scratch.summary()

print("\nĐang huấn luyện Mô hình 4...")
lstm_model_scratch.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

lstm_model_scratch.fit(
    X_train_pad, y_train,
    validation_data=(X_val_pad, y_val),
    epochs=20,
    batch_size=32,
    callbacks=[early_stopping],
    shuffle=True
)

print("\n--- [Nhiệm vụ 4] Kết quả trên tập Test ---")
loss_lstm_scratch, acc_lstm_scratch = lstm_model_scratch.evaluate(X_test_pad, y_test)
print(f"Test Loss: {loss_lstm_scratch:.4f}, Test Accuracy: {acc_lstm_scratch:.4f}")

y_pred_lstm_scratch = lstm_model_scratch.predict(X_test_pad)
y_pred_lstm_scratch_classes = np.argmax(y_pred_lstm_scratch, axis=1)
print(classification_report(y_test, y_pred_lstm_scratch_classes, target_names=label_encoder.classes_))

f1_lstm_scratch = f1_score(y_test, y_pred_lstm_scratch_classes, average='macro')


Đã Tokenize và Pad. Shape X_train_pad: (8954, 50)


--- [Nhiệm vụ 3] Đang thực hiện Embedding Pre-trained + LSTM ---
Đã tạo Embedding Matrix. Shape: (4265, 100)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │       426,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 426,500 (1.63 MB)

...

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


Đang huấn luyện Mô hình 4...
Epoch 1/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 51s 171ms/step - accuracy: 0.0167 - loss: 4.1513 - val_accuracy: 0.0177 - val_loss: 4.1278
Epoch 2/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 47s 167ms/step - accuracy: 0.0147 - loss: 4.1381 - val_accuracy: 0.0177 - val_loss: 4.1290
Epoch 3/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 48s 170ms/step - accuracy: 0.0147 - loss: 4.1330 - val_accuracy: 0.0177 - val_loss: 4.1254
Epoch 4/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 48s 170ms/step - accuracy: 0.0149 - loss: 4.1316 - val_accuracy: 0.0177 - val_loss: 4.1266
Epoch 5/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 46s 165ms/step - accuracy: 0.0183 - loss: 4.1319 - val_accuracy: 0.0177 - val_loss: 4.1255
Epoch 6/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 83s 171ms/step - accuracy: 0.0160 - loss: 4.1313 - val_accuracy: 0.0177 - val_loss: 4.1253
Epoch 7/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 46s 166ms/step - accuracy: 0.0151 - loss: 4.1321 - val_accuracy: 0.0177 - val_loss: 4.1245
Epoch 8/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 48s 170ms/ste

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [12]:
import pandas as pd
import numpy as np

# 1. PHÂN TÍCH ĐỊNH LƯỢNG (BẢNG SO SÁNH)

results = {
    "Pipeline": [
        "TF-IDF + Logistic Regression",
        "Word2Vec (Avg) + Dense",
        "Embedding (Pre-trained) + LSTM",
        "Embedding (Scratch) + LSTM"
    ],
    "F1-score (Macro)": [
        f1_lr,
        f1_w2v_avg,
        f1_lstm_pre,
        f1_lstm_scratch
    ],
    "Test Loss": [
        loss_lr,
        loss_w2v_avg,
        loss_lstm_pre,
        loss_lstm_scratch
    ]
}

results_df = pd.DataFrame(results)
print("--- BẢNG SO SÁNH KẾT QUẢ ĐỊNH LƯỢNG ---")
print(results_df.to_markdown(index=False, floatfmt=".4f"))

# 2. PHÂN TÍCH ĐỊNH TÍNH (CÂU KHÓ)

print("\n\n--- PHÂN TÍCH ĐỊNH TÍNH (CÁC CÂU KHÓ) ---")

test_sentences = [
    "can you remind me to not call my mom",
    "is it going to be sunny or rainy tomorrow",
    "find a flight from new york to london but not through paris",
    "what alarms do i have set right now"
]
def get_all_predictions(text_list):
    pred_lr = tfidf_lr_pipeline.predict(text_list)

    avg_vecs = np.array([sentence_to_avg_vector(text, w2v_model) for text in text_list])
    pred_w2v_nn = np.argmax(model_w2v_avg.predict(avg_vecs), axis=1)

    seqs = tokenizer.texts_to_sequences(text_list)
    padded = pad_sequences(seqs, maxlen=max_len, padding='post')

    pred_lstm_pre = np.argmax(lstm_model_pretrained.predict(padded), axis=1)
    pred_lstm_scratch = np.argmax(lstm_model_scratch.predict(padded), axis=1)

    return {
        "TF-IDF": label_encoder.inverse_transform(pred_lr),
        "W2V (Avg)": label_encoder.inverse_transform(pred_w2v_nn),
        "LSTM (Pre)": label_encoder.inverse_transform(pred_lstm_pre),
        "LSTM (Scratch)": label_encoder.inverse_transform(pred_lstm_scratch)
    }

# Lấy dự đoán cho các câu mẫu
predictions = get_all_predictions(test_sentences)

# In kết quả
for i, sentence in enumerate(test_sentences):
    print(f"\nCâu: '{sentence}'")
    print(f"  - Dự đoán [TF-IDF]:        {predictions['TF-IDF'][i]}")
    print(f"  - Dự đoán [W2V (Avg)]:     {predictions['W2V (Avg)'][i]}")
    print(f"  - Dự đoán [LSTM (Pre)]:   {predictions['LSTM (Pre)'][i]}")
    print(f"  - Dự đoán [LSTM (Scratch)]: {predictions['LSTM (Scratch)'][i]}")

--- BẢNG SO SÁNH KẾT QUẢ ĐỊNH LƯỢNG ---
| Pipeline                       |   F1-score (Macro) |   Test Loss |
|:-------------------------------|-------------------:|------------:|
| TF-IDF + Logistic Regression   |             0.8353 |    nan      |
| Word2Vec (Avg) + Dense         |             0.1317 |      3.1565 |
| Embedding (Pre-trained) + LSTM |             0.0640 |      3.3827 |
| Embedding (Scratch) + LSTM     |             0.0005 |      4.1238 |


--- PHÂN TÍCH ĐỊNH TÍNH (CÁC CÂU KHÓ) ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 296ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step

Câu: 'can you remind me to not call my mom'
  - Dự đoán [TF-IDF]:        calendar_set
  - Dự đoán [W2V (Avg)]:     general_explain
  - Dự đoán [LSTM (Pre)]:   general_negate
  - Dự đoán [LSTM (Scratch)]: play_radio

Câu: 'is it going to be sunny or rainy tomorrow'
  - Dự đoán [TF-IDF]:        weather_query
  - Dự đoán [W2V (Avg)]:     calendar_query
  - Dự đoán [LSTM (Pre)]:   

## Tóm tắt kết quả

* **Mô hình 1 (TF-IDF + Logistic Regression):** Đạt hiệu suất cao nhất với **Accuracy 84%** và F1-Macro 0.84.
* **Mô hình 2 (Word2Vec Trung bình):** Accuracy 19% và F1-Macro 0.13.
* **Mô hình 3 (LSTM Pre-trained):** Accuracy 11% và F1-Macro 0.06.
* **Mô hình 4 (LSTM Học từ đầu):** Accuracy 1.7% và F1-Macro 0.00 (gần như đoán mò).

---

## Đánh giá kết quả

Kết quả gây bất ngờ là mô hình "túi từ" cổ điển (TF-IDF) đã cho ra kết quả tốt nhất. Các mô hình nơ-ron phức tạp hơn (Word2Vec Avg, LSTM) đều thất bại hoàn toàn.

Điều này chứng tỏ sức mạnh lý thuyết của LSTM (hiểu ngữ cảnh, trật tự từ) đã không được thể hiện trong bài lab này. Ngược lại, phương pháp dựa trên từ khóa (TF-IDF) lại tỏ ra hiệu quả vượt trội.

---

## Dự đoán và phân tích nguyên nhân (Giả thuyết)

1.  **Mô hình 1 (TF-IDF)**  có thể là do bài toán này dựa nhiều vào **từ khóa**. TF-IDF rất giỏi trong việc bắt các từ khóa này nên đã cho kết quả cao.

2.  **Mô hình 2 (Word2Vec)** có thể là do việc **lấy vector trung bình** đã làm "mờ" đi các từ khóa quan trọng. Biểu diễn câu bị "pha loãng" khiến mô hình không còn phân biệt được nữa.

3.  **Mô hình 3 (LSTM Pre-trained)** có thể là do nguồn "pre-trained" (từ `w2v_model`) của mình **chất lượng quá thấp**. Nó chỉ học từ 9k câu, nên vector không tốt.

4.  **Mô hình 4 (LSTM Scratch)** có lẽ là do **dữ liệu quá nhỏ (8954 mẫu)** so với một mô hình phức tạp (LSTM) và bài toán quá khó (64 lớp). Mô hình dường như đã không thể hội tụ và bị "kẹt" ở một điểm tối ưu cục bộ.